In [47]:
import csv # for writing dataframes to csv
import random # for making a random choice
import os # for scanning directories
import itertools
import string # for generating strings
from collections import Counter

import kintypes as kt # bringing large lists of kin types into the namespace
import math # for calculating logs
import pandas as pd
import numpy as np
import scipy.stats
import re
from tqdm import tqdm

testing = True # set to True to run code blocks with tests and examples
filtering = False # set to True to run the filtering process

# Internal co-selection

Internal co-selection refers to a process of kin term evolution whereby terminological changes in one part of the paradigm co-occur with changes in related parts of the paradigm, increasing the predictive structure of the paradigm.

In this notebook, we will build simulations to investigate the robustness of this tendency cross-linguistically, using data from Kinbank, a global database of kin terminology. 

We will measure internal co-selection in terms of the **mutual information** between Generation N and Generation N+1 in a particular kinship system. That tells us how much information can be gained from one generation by observing the other - how certain we can be about which children 'go with' which parents. This can be calculated as the **entropy** of one generation (how much unpredictable variation there is) minus the **conditional entropy** between the two generations (how much unpredictability remains in one generation after observing another).

## Calculating mutual information of a kinship system

To calculate the mutual information (MI) of a particular kinship system, we must perform the following steps:

1. Extract kin terminology data from Kinbank for this language.
2. Condense the full kinship system down to the terms we are interested in: Ego's generation and Ego's parents' generation.
3. Calculate the probabilities of each kin term within the generation in which it belongs; and the probabilities of each parent-child pair.
4. Calculate entropy, conditional entropy, subtract them from each other to get the mutual information of the system.

After we get that going, we can do these same calculations on simulated kinship systems.

### 1. Extract kin terminology from Kinbank

First, let's actually load our data in. The following function `get_kb_files()` pulls the full list of Kinbank filenames - one file per language. Later, we can iterate through these to generate MI values for every language in our dataset.

In [48]:
def get_kb_files(path) -> list:
    files = []
    directory = os.scandir(path)
    for file in directory:
        files.append(file.name)
    return files

all_kb_files = get_kb_files('../../kinbank')

In [49]:
all_kb_files

["'Are'are_area1240.csv",
 'Abui_abui1241.csv',
 'Achinese_achi1257.csv',
 'Achuar_achu1249.csv',
 'Aché_ache1246.csv',
 'Adang_adan1251.csv',
 'Adnyamathanha_adny1235.csv',
 'Aghul_aghu1253.csv',
 'Agob-Ende-Kawam_agob1244.csv',
 'Aguaruna_agua1253.csv',
 'Akhvakh_Northern_Akhvakh__akhv1239.csv',
 'Akkadian_akka1240.csv',
 'Aklanon_akla1241.csv',
 'Alamblak_alam1246.csv',
 'Alangan_alan1249.csv',
 'Albanian_alba1267.csv',
 'Albanian_Tosk__tosk1239.csv',
 'Alyawarr_alya1239.csv',
 'Amanab_aman1265.csv',
 'Amharic_amha1245.csv',
 'Amis_amis1246.csv',
 'Ampibabo_Lauje_ampi1237.csv',
 'Anambé_anam1249.csv',
 'Ancient_Greek_anci1242.csv',
 'Andi_andi1255.csv',
 'Anejom_anei1239.csv',
 'Angaite_anga1316.csv',
 'Aniwa_aniw1237.csv',
 'Anuta_anut1237.csv',
 'Arabana_arab1267.csv',
 'Arabana_arab1267a.csv',
 'Araki_arak1252.csv',
 'Araweté_araw1273.csv',
 'Archi_arch1244.csv',
 'Aria_(Mouk-Aria)_mouk1240a.csv',
 'Arizona_Yaqui_yaqu1251a.csv',
 'Armenian_nucl1235.csv',
 'Arosi_aros1241.csv',
 '

Using one of these filenames, we can extract the kin terminology from that file and populate a dictionary with it. We're only interested in two columns from the Kinbank data: `parameter`, which contains a short code indicating a **kin type**, and `word`, which contains the **kin term** associated with that kin type. An example of a row in the English data would be `mMeB, uncle`, where `mMeB` means 'male speaker's mother's older brother', and `uncle` is the term associated with that person.

In [50]:
def get_kin_terms(filepath: str) -> dict:
    ks = {}
    with open(filepath, encoding='utf8') as f:
        csv_reader = csv.DictReader(f)
        next(csv_reader) # to skip the header row
        for line in csv_reader:
            kin_type = line['parameter']
            kin_term = line['word']
            ks[kin_type] = kin_term
    return ks

### 2. Condense the system down

We're interested in the mutual information between the kin terms in Generation 0 (Ego's generaiton) and Generation +1 (Ego's parents' generation).

We want create a data structure that pairs up parent types with the corresponding child types. This is because we're interested in whether kinship systems maintain patterns of terminological distinctions and mergers across these two generations, so we will need to know which parent terms 'go with' which child terms.

In `kintypes`, you will find a list of pairs of kin types, where the first element in the pair is a parent type, and the second is their child; e.g. mMeB and mMeBD (mother's elder brother and mother's elder brother's daughter). 

`get_pairs()` takes a kinship system as input, and outputs a list of tuples. The first element in the tuple is the parent term, the second is the corresponding child term. 

In [51]:
def get_pairs(ks: dict) -> list:
    pairs_of_terms = []
    parent_types = []

    for pair in kt.ics_pairs:
        if pair[0] in ks and pair[1] in ks:
            pairs_of_terms.append((ks[pair[0]],ks[pair[1]]))
            parent_types.append(pair[0])
                
    return pairs_of_terms

But for our calculations, we'll still need to know which terms belong to which generation. Luckily, we know that the 0th element in each tuple is from Ego's parents' generation and the 1st element is from Ego's generation. So we can happily split these tuples down the middle and populate two lists with the terms. `split_pairs()` takes a list of pairs and sorts it into terms that belong to Ego's generation and terms that belong to Ego's parents' generation.

In [52]:
def split_pairs(pairs: list) -> list:
    gn = []
    gn1 = []
    for pair in pairs:
        gn.append(pair[1])
        gn1.append(pair[0])
    
    return gn,gn1

### 3. Calculate probabilities

To calculate entropy, we need a probability distribution over the terms in one single generation of a kinship system. So let's start with a function that can calculate the probability of a particular term.

Given a term and the full list of terms in the same generation, `probability()` counts how many times that term exists in `generation` and divides that by the total length of `generation`.

In [53]:
def probability(term: str, generation: list) -> float:
    return generation.count(term)/len(generation)

When calculating mutual information, we also need the **conditional entropy** between the two generations of our system. To calculate this, we will need not only the probabilities of terms in a generation, but also the **joint probabilities** of every pair of terms across those two generations. In other words, we need to calculate the probabilities of our `get_pairs` output.

Given two terms, `joint_probability()` counts how many pairs made of those two terms exist in `pairs`, then divides that by the total length of `pairs`.

In [54]:
def joint_probability(term1: str, term2: str, pairs: list) -> float:
    pair = (term1,term2)
    return pairs.count(pair)/len(pairs)

### 4. Calculating entropy and mutual information

Entropy (in bits) is defined as 

$$
H(X) = -\sum_{x \in X}p(x) log_2p(x)
$$

or, in English, it is the inverse sum over a distribution X of the probability of y * the log probability of y.

Entropy is a measure of the average level of uncertainty about the possible outcomes of a variable.

First, let's define a function `entropy()` that will iterate over a generation of the kinship system and output the entropy of that generation. 

Note: we only need one generation's entropy score to calculate mutual information - we make the arbitrary choice to calculate the entropy of Ego's parents' generation later in this notebook.

In [55]:
def entropy(generation: list) -> list:
    entropy = 0
    for term in set(generation): # using a set as we want to count each unique term only once
        p = probability(term,generation)
        #print('entropy of',term,p*math.log(p))
        entropy += p*math.log2(p)
    return round(-entropy,5)

Conditional entropy of Y given X is defined as

$$
H(Y|X) = -\sum_{x \in X,y \in Y}p(x,y) log_2 {p(x,y) \over p(x)}
$$

or in English, the inverse sum over two distributions Y and X of the probability of each y * the log probability of each y given x.

Conditional entropy is the amount of information needed to describe the outcome of a random variable Y given that we already know the value of another random variable X.

To calculate it, we need the joint probability of each pair (given by `joint_probability()`) and the probability of one member of that pair (given by `probability()`). We can then calculate the conditional probability of parent term given child term as the joint probability of those terms over the probability of the parent term.

`conditional_entropy()` iterates over all pairs to output the conditional entropy of Ego's generation given Ego's parents' generation.

In [56]:
def conditional_entropy(gn: list, pairs:list) -> float:
    entropy = 0
    for x,y in set(pairs): # x = parent, y = child
        p_xy = joint_probability(x,y,pairs)
        p_y = probability(y,gn)
        if p_xy > 0 and p_y > 0:
            #print('p(', x, '|', y,') = ', p_xy/p_y, 'p(y) = ', p_y)
            entropy += p_xy * math.log2(p_xy/p_y)
    return round(-entropy,5)

Finally, mutual information is defined as

$$
I(X;Y) \equiv H(X) - H(X|Y)
$$

or in English, entropy of X minus the conditional entropy of X given Y.

In this study, it is equal to the entropy of Ego's parents' generation minus the conditional entropy of Ego's parents' generation given Ego's generation. It tells us how much mutual dependence there is between these two generations; i.e. how much we can predict about one by observing the other.

In [57]:
def mutual_information(pairs: list):
    gn,gn1 = split_pairs(pairs)
    e = entropy(gn1)
    ce = conditional_entropy(gn,pairs)
    mi = e - ce
    return round(mi,5)

### 5. Calculate MI for each language and save data

With all of the above infrastructure, we can calculate the mutual information between G0 and G+1 for all the languages in Kinbank and save this data to a separate .csv file.

First, a function that calculates the relevant data and stores it.

In [58]:
def write_data(pairs,results):
    gn,gn1 = split_pairs(pairs)
    egn = entropy(gn)
    egn1 = entropy(gn1)
    ce = conditional_entropy(gn,pairs)
    mi = mutual_information(pairs)
    
    results['mutual_information'] = mi
    results['entropy_gn'] = egn
    results['entropy_gn1'] = egn1
    results['conditional_entropy'] = ce
    results['variation_gn'] = len(set(gn))
    results['variation_gn1'] = len(set(gn1))
    results['number_of_pairs'] = len(set(pairs))
    
    return results

In [59]:
families = ['Afro-Asiatic','Algic','Arawakan','Atlantic-Congo','Austroasiatic','Austronesian','Cariban','Dravidian','Indo-European',
            'Nakh-Daghestanian','Nuclear Trans New Guinea', 'Other','Pama-Nyungan','Pano-Tacanan','Salishan','Sino-Tibetan',
            'Tai-Kadai','Tupian','Turkic','Uralic','Uto-Aztecan']



In [60]:
def calculate_MI(families: list,filename: str):
    df = []
    codes = []
    filepath = '../../kinbank-family/'
    
    for family in families:
        all_files = get_kb_files(filepath + family)
        
        for file in all_files:
            print(file)
            match = re.search("[a-z]{4}[0-9]{4}[a-z]?", file)
            code = match.group()
            language = file.split('_' + code)[0]

            if code not in codes:
                codes.append(code)

                ks = get_kin_terms(filepath + family + '/' + file)

                pairs = get_pairs(ks)

                if pairs: # if pairs is not empty

                    results = {}
                    results['language'] = language
                    results['language_family'] = family
                    results['code'] = code
                    results['simulation_code'] = code + '_REAL'
                    results['simulation'] = 'N'
                    write_data(pairs,results)

                    df.append(results)
        
    pd.DataFrame(df).to_csv('../data/raw/' + filename + '.csv',index=False)
    
    return pd.DataFrame(df)

In [22]:
calculate_MI(['Indo-European'],'test')

Albanian_alba1267.csv
Albanian_Tosk__tosk1239.csv
Ancient_Greek_anci1242.csv
Armenian_nucl1235.csv
Assamese_assa1263.csv
Avestan_aves1237.csv
Balochi_balo1260.csv
Belarusian_bela1254.csv
Bengali_beng1280.csv
Bihari_biha1245.csv
Boers_afri1274.csv
Breton_bret1244.csv
Bulgarian_bulg1262.csv
Central_Kurdish_cent1972.csv
Chavacano_chav1241.csv
Cornish_corn1251.csv
Czech_czec1258.csv
Danish_dani1285.csv
Dari_dari1249.csv
Dutch_dutc1256.csv
Eastern_Armenian_east2283.csv
Eastern_Pahari_Nepali__east1436.csv
Eastern_Yiddish_east2295.csv
English_stan1293.csv
Frisian_fris1239.csv
German_stan1295.csv
Gothic_goth1244.csv
Gujarati_guja1252.csv
Hindi_hind1269.csv
Hittite_hitt1242.csv
Icelandic_icel1247.csv
Irish_iris1253.csv
Italian_ital1282.csv
Jamaican_Creole_jama1262.csv
Judeo_Tat_jude1256.csv
Kashmiri_kash1277.csv
Kriol_Roper_River_krio1252.csv
Kristang_(Malacca_Creole_Portugese)_mala1533.csv
Kumaoni_kuma1273.csv
Lahnda_lahn1241.csv
Latin_lati1261.csv
Latin_lati1261a.csv
Latvian_latv1249.csv
Lith

,language,language_family,code,simulation_code,simulation,mutual_information,entropy_gn,entropy_gn1,conditional_entropy,variation_gn,variation_gn1,number_of_pairs
0,Albanian,Indo-European,alba1267,alba1267_REAL,N,0.91829,2.25163,2.58496,1.66667,6,6,16
1,Albanian_Tosk_,Indo-European,tosk1239,tosk1239_REAL,N,0.91829,1.41830,2.58496,1.66667,4,6,10
2,Ancient_Greek,Indo-European,anci1242,anci1242_REAL,N,0.91830,1.91830,1.91830,1.00000,4,4,8
3,Armenian,Indo-European,nucl1235,nucl1235_REAL,N,2.25163,3.25163,2.58496,0.33333,10,6,12
4,Assamese,Indo-European,assa1263,assa1263_REAL,N,1.50000,2.25000,2.62500,1.12500,5,7,11
...,...,...,...,...,...,...,...,...,...,...,...,...
96,Vlax_Romani,Indo-European,vlax1238,vlax1238_REAL,N,0.91830,1.91830,1.91830,1.00000,4,4,8
97,Wallon,Indo-European,wall1255,wall1255_REAL,N,0.91829,1.91830,2.18872,1.27043,4,5,10
98,Western_Armenian,Indo-European,homs1234,homs1234_REAL,N,0.91829,1.58496,2.58496,1.66667,5,6,12
99,Western_Farsi,Indo-European,west2369,west2369_REAL,N,2.22987,3.57770,2.57770,0.34783,12,6,16


In [61]:
calculate_MI(families,'kinbank_mi_UPDATED_NEW_KINTYPES')

Akkadian_akka1240.csv
Amharic_amha1245.csv
Hausa_haus1257.csv
Lebanese_Arabic_stan1323.csv
Old_Aramaic_olda1245.csv
Polci_polc1243.csv
Morgan1871_An-ah'-ne-lin_(Gros_Ventres_of_Prairie)_gros1243.csv
Morgan1871_Blood_Siksika_(Blood_Blackfoot)_bloo1239.csv
Morgan1871_Cree_of_the_Lowlands_cree1272.csv
Morgan1871_Delaware_comm1246.csv
Morgan1871_Etchemiu,_or_Malisete_male1293.csv
Morgan1871_Kaskaskia_algo1255.csv
Morgan1871_Kickapoo_(Kikapoo)_kick1244.csv
Morgan1871_Menomine_meno1252.csv
Morgan1871_Miami_miam1252.csv
Morgan1871_Miemac_mikm1235.csv
Morgan1871_Mohegan_mohe1244.csv
Morgan1871_Munsee_muns1251.csv
Morgan1871_Ojibwa_Kansas_ojib1241.csv
Morgan1871_Ojibwa_Lake_Huron_ojib1241.csv
Morgan1871_Ojibwa_Lake_Michigan_uppe1274.csv
Morgan1871_Ojibwa_Lake_Superior_ojib1241.csv
Morgan1871_Ottawa_(Otǎwa)_otta1242.csv
Morgan1871_Peoria_peor1238.csv
Morgan1871_Pianeshaw_miam1252.csv
Morgan1871_Piegan_Siksika_(Piegan_Blackfoot)_pieg1239.csv
Morgan1871_Plains_Cree_(Cree_of_the_Prairie)_plai1258.c

Seke_(Ske)_seke1241.csv
Senbarei_unua1237.csv
Sie_(Sye)_siee1239.csv
Sikaiana_sika1261.csv
Sika_sika1262.csv
Sonsorol_sons1242.csv
Southeast_Ambrym_sout2859.csv
Southeast_Babar_sout2883.csv
Southwest_Tanna_sout2869.csv
South_Efate_sout2856a.csv
South_Marquesan_sout2866.csv
South_Tuvaluan_(Vaitupu)_sout2865.csv
Sula_Fagudu_sula1245.csv
Sula_Mangoli_mang1408.csv
Sundanese_sund1252.csv
Sungagage_mari1426i.csv
Sungwadaga_mari1426b.csv
Sungwadia_mari1426a.csv
Sungwaloge_(Nalemba_Edward)_mari1426g.csv
Sungwaloge_(Nalemba_Simeone_Tari)_mari1426f.csv
Sungwaloge_(Navenvene)_mari1426d.csv
Sungwaloge_(Tauta)_mari1426e.csv
Sungwaloge_(Tawet)_mari1426c.csv
Sunwadia_mari1426.csv
Tadyawan_tady1237.csv
Tagalog_taga1270.csv
Tahitians_tahi1242.csv
Tahitian_tahi1242.csv
Tajio_taji1246.csv
Takia_taki1248.csv
Takuu_taku1257.csv
Talibabu_tali1262.csv
Talise_(Tolo)_tali1259.csv
Tamambo_malo1243.csv
Tanala_tana1285.csv
Tanampedagi_Taje_taje1237b.csv
Tangoa_(Movono)_tang1347.csv
Tasiko_tasi1237.csv
Taumako_tau

Sougb_mani1235.csv
Southern_Coastal_Tsimshian_nucl1649.csv
Suki_suki1245.csv
Tabla_tabl1243.csv
Taiap_taia1239.csv
Tehuelche_tehu1242.csv
Teiwa_teiw1235.csv
Telefol_tele1256.csv
Tidore_(version_1)_tido1248.csv
Tidore_(version_2)_tido1248a.csv
Tidore_tido1249.csv
Tlingit_tlin1245.csv
Toba_toba1269.csv
Tolowa_tolo1259.csv
Touo_touo1238.csv
Trumai_trum1247.csv
Tsafiki_colo1249.csv
Tucanoan_Bará_waim1255.csv
Tucano_tuca1252.csv
Tuyuca_tuyu1244.csv
Tzeltal_tzel1254.csv
Umbu-Ungu_umbu1258.csv
Usurufa_usar1243.csv
Waffa_waff1241.csv
Wailaki_wail1244.csv
Wambaya_wamb1258.csv
Waorani_waor1240.csv
Wappo_wapp1239.csv
Wari__wari1268.csv
Warta_Thuntai_gunt1241.csv
Watam_wata1253.csv
Wersing_wers1238.csv
Western_Apache_west2615.csv
Western_Pantar_lamm1241.csv
Wichí_wich1264.csv
Wipi_wipi1242.csv
Xibe_xibe1242.csv
Yale_kosa1249.csv
Yalic_yali1257.csv
Yamana_yama1264.csv
Yanomámi_yano1262.csv
Yawa_nucl1454.csv
Yele_(Yélî_Dnye)_yele1255.csv
Yelmek_yelm1242.csv
Yucatec_Maya_yuca1254.csv
Yuki_yuki1243.cs

,language,language_family,code,simulation_code,simulation,mutual_information,entropy_gn,entropy_gn1,conditional_entropy,variation_gn,variation_gn1,number_of_pairs
0,Akkadian,Afro-Asiatic,akka1240,akka1240_REAL,N,0.00000,2.25000,1.00000,1.00000,5,2,10
1,Amharic,Afro-Asiatic,amha1245,amha1245_REAL,N,1.58497,2.25163,1.91830,0.33333,6,4,10
2,Hausa,Afro-Asiatic,haus1257,haus1257_REAL,N,0.58496,1.58496,2.25163,1.66667,5,5,12
3,Lebanese_Arabic,Afro-Asiatic,stan1323,stan1323_REAL,N,1.58219,2.57770,2.58219,1.00000,6,6,12
4,Old_Aramaic,Afro-Asiatic,olda1245,olda1245_REAL,N,1.00000,2.00000,2.00000,1.00000,5,4,10
...,...,...,...,...,...,...,...,...,...,...,...,...
1067,Tubar,Uto-Aztecan,tuba1279,tuba1279_REAL,N,0.00000,1.52193,-0.00000,-0.00000,3,1,3
1068,Tübatülabal,Uto-Aztecan,tuba1278,tuba1278_REAL,N,0.00000,0.91830,1.00000,1.00000,2,2,4
1069,Tümpisha_Shoshoni,Uto-Aztecan,pana1305,pana1305_REAL,N,0.88654,2.17574,2.58219,1.69565,6,6,16
1070,White_Mesa_Ute,Uto-Aztecan,utee1244b,utee1244b_REAL,N,0.00000,-0.00000,-0.00000,-0.00000,1,1,1


## Simulating kinship systems

To investigate whether kinship systems have higher mutual information than chance, we can build a random baseline for each language to serve as a point of comparison.

To do this, we will take each language in our dataset, and randomly scramble which terms go with which relatives (within generations). This will randomise the syncretisms within the paradigm, breaking any predictable structure built by the internal co-selection process, while maintaining the amount of variation across the system overall.
 
We take the following steps:

1. Extract the kinship system of a language from kinbank.
2. Filter the two generations we are interested in.
3. Randomly reassign the kinship terms to new types.
4. Repeat the process 1000 times for each language.

We already have the infrastructure for the first two! `get_kin_terms()`,  `get_pairs()` and `split_pairs()` will do this for us. So let's skip to 3, and write a function that randomises which terms form pairs, assuming that we have already extracted the kinship system and filtered the relevant pairs.

### 3. Randomly rearrange kin terms

`kintypes.py` also contains a list of which kin types are in which generation, which we can use to split a kinship system by generation.

In [62]:
def split_ks(ks):
    gn = {}
    gn1 = {}
    for entry in ks:
        if entry in kt.generation_n:
            gn[entry] = ks[entry]
        elif entry in kt.generation_n1:
            gn1[entry] = ks[entry]
        else:
            pass

    return gn,gn1        

Once the kinship system is split, we can then shuffle one of the generations and recombine the system.

In [63]:
def shuffle_ks(ks):
    gn_terms = []
    gn,gn1 = split_ks(ks)
    for term in gn:
        gn_terms.append(gn[term])

    random.shuffle(gn_terms)
    
    for i in range(len(gn)):
        # print(i)
        key = list(gn.keys())[i]
        ks[key] = gn_terms[i]

    return ks

If we split the kinship system, shuffle the G0 terms, and stick the system back together, then we can use `get_pairs()` just as we did for the real kinship systems to filter out the two generations we're interested in. From there, we can calculate MI for each simulation and save the data in a similar way as before.

In [64]:
def simulate_ks(ks: dict) -> list:
    simulation = shuffle_ks(ks)
    shuffled_pairs = get_pairs(simulation)
    return shuffled_pairs

In [65]:
def simulate_MI(families: list, filename: str, times: int):
    df = []
    codes = []
    filepath = '../../kinbank-family/'
    
    for family in tqdm(families):
        all_files = get_kb_files(filepath + family)
    
        for file in all_files:
            match = re.search('[a-z]{4}[0-9]{4}[a-z]?', file)
            code = match.group()
            language = file.split('_' + code)[0]

            if code not in codes:
                codes.append(code)

                ks = get_kin_terms(filepath + family + '/' + file)

                for i in range(times):
                    pairs = simulate_ks(ks)
                    if pairs:
                        results = {}
                        results['language'] = language
                        results['language_family'] = family
                        results['code'] = code
                        results['simulation_code'] = code + '_' + str(i)
                        results['simulation'] = 'Y'
                        write_data(pairs,results)

                        df.append(results)
    
    pd.DataFrame(df).to_csv('../data/raw/' + filename + '.csv',index=False)
    
    return pd.DataFrame(df)

In [20]:
simulate_MI(['Indo-European'],'test',10)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.17s/it]


,language,language_family,code,simulation_code,simulation,mutual_information,entropy_gn,entropy_gn1,conditional_entropy,variation_gn,variation_gn1,number_of_pairs
0,Albanian,Indo-European,alba1267,alba1267_0,Y,0.34944,2.00092,2.58496,2.23552,6,6,23
1,Albanian,Indo-European,alba1267,alba1267_1,Y,0.38965,1.87863,2.58496,2.19531,6,6,19
2,Albanian,Indo-European,alba1267,alba1267_2,Y,0.33172,1.87863,2.58496,2.25324,6,6,21
3,Albanian,Indo-European,alba1267,alba1267_3,Y,0.42473,1.81519,2.58496,2.16023,6,6,18
4,Albanian,Indo-European,alba1267,alba1267_4,Y,0.29059,1.87863,2.58496,2.29437,6,6,21
...,...,...,...,...,...,...,...,...,...,...,...,...
1005,Western_Panjabi,Indo-European,west2386,west2386_5,Y,0.26953,1.14890,2.58496,2.31543,5,6,16
1006,Western_Panjabi,Indo-European,west2386,west2386_6,Y,0.30576,1.06651,2.58496,2.27920,5,6,14
1007,Western_Panjabi,Indo-European,west2386,west2386_7,Y,0.26912,1.06651,2.58496,2.31584,5,6,15
1008,Western_Panjabi,Indo-European,west2386,west2386_8,Y,0.32253,0.98336,2.58496,2.26243,5,6,13


In [66]:
simulate_MI(families,'simulated_mi_UPDATED_NEW_KINTYPES',1000)

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [03:58<00:00, 11.34s/it]


,language,language_family,code,simulation_code,simulation,mutual_information,entropy_gn,entropy_gn1,conditional_entropy,variation_gn,variation_gn1,number_of_pairs
0,Akkadian,Afro-Asiatic,akka1240,akka1240_0,Y,0.0,2.25000,1.0,1.0,5,2,10
1,Akkadian,Afro-Asiatic,akka1240,akka1240_1,Y,0.0,2.25000,1.0,1.0,5,2,10
2,Akkadian,Afro-Asiatic,akka1240,akka1240_2,Y,0.0,2.25000,1.0,1.0,5,2,10
3,Akkadian,Afro-Asiatic,akka1240,akka1240_3,Y,0.0,2.25000,1.0,1.0,5,2,10
4,Akkadian,Afro-Asiatic,akka1240,akka1240_4,Y,0.0,2.25000,1.0,1.0,5,2,10
...,...,...,...,...,...,...,...,...,...,...,...,...
1071995,Yaqui,Uto-Aztecan,yaqu1251,yaqu1251_995,Y,0.0,1.37878,1.0,1.0,3,2,6
1071996,Yaqui,Uto-Aztecan,yaqu1251,yaqu1251_996,Y,0.0,1.84237,1.0,1.0,4,2,8
1071997,Yaqui,Uto-Aztecan,yaqu1251,yaqu1251_997,Y,0.0,1.95021,1.0,1.0,4,2,8
1071998,Yaqui,Uto-Aztecan,yaqu1251,yaqu1251_998,Y,0.0,1.84237,1.0,1.0,4,2,8


## Edit Distance

In this section we calculate the normalised Levenshtein edit distance between pairs of kin terms in each language and generate a monte carlo sample of edit distance for each language so that we can calculate the z score (as well as the correlation between edit distances).

To do this we need:
* A function to calculate the levenshtein distance between two forms.
* A function to calculate the distance between two meanings, which should have e.g. MB and MBD be closer than MZ and MBD.
* A function to calculate the correlation between a table of form distances and a table of meaning distances.
* A function to generate a monte carlo sample of correlations (ie the correlation of scrambled, simulated data).


First, to calculate edit distance:

In [21]:
from Levenshtein import distance as lvs_dist

In [22]:
x = 'aabb'
y = 'bbbb'

lvs_dist(x,y)/len(max(x,y))


print(len(max(x,y)) == max(len(x),len(y)))


True


How about for actual language data?

In [23]:
test_file = random.choice(all_kb_files)

test = get_kin_terms('../../kinbank/' + test_file)

english = get_kin_terms('../../kinbank/English_stan1293.csv')

In [24]:
test_gn,test_gn1 = split_ks(test)

test_ks = {**test_gn, **test_gn1}

eng_gn,eng_gn1 = split_ks(english)
eng_ks = {**eng_gn, **eng_gn1}

In [25]:
def edit_distance(ks):
#     df = []
    edit_dists = []

    for relative1 in ks:
        term1 = ks[relative1]
        for relative2 in ks:
            data = {}
            term2 = ks[relative2]
            
            if relative1 == relative2:
                pass
            
            else:
            
                if len(term1) > 0 and len(term2) > 0:
                    dist = lvs_dist(term1,term2)/len(max(term1,term2))
                    edit_dists.append(dist)
                else:
                    pass
                    
                
#                 data['relative1'] = relative1
#                 data['relative2'] = relative2
#                 data['label1'] = term1
#                 data['label2'] = term2
#                 data['edit_distance'] = dist

#                 df.append(data)

#     distance_df = pd.DataFrame(df)

#     return distance_df
    return edit_dists

To calculate semantic distance:

In [26]:
pairs = kt.ics_pairs
relatives = []

for pair in pairs:
    relatives.append(pair[0])
    relatives.append(pair[1])
    
relatives = list(set(relatives))

In [27]:
def semantic_distance(ks,pairs): # (df,ks,pairs):
    sem_distances = []

    for relative1 in ks:
        for relative2 in ks:
            if relative1 == relative2:
                pass
            
            else:
            
                if len(ks[relative1]) > 0 and len(ks[relative2]) > 0:

                    if (relative1,relative2) in pairs or (relative2,relative1) in pairs:
                        distance = 1
                        sem_distances.append(distance)

                    else:
                        distance = 2
                        sem_distances.append(distance)

#     df['semantic_distance'] = sem_distances
#     
#     return df
    return sem_distances


In [28]:
# scipy.stats.pearsonr(np.array(distance_df['edit_distance']),np.array(distance_df['semantic_distance']))

ed = edit_distance(test_ks)
semd = semantic_distance(test_ks,pairs)

print(len(ed),len(semd))

scipy.stats.pearsonr(np.array(edit_distance(test_ks)),np.array(semantic_distance(test_ks,pairs)))

702 702


(-0.2191545123436959, 4.419112045385183e-09)

The above will get us a meaning distance measure where if you are the child of a person, your meaning distance will be 1, and if you are not, your distance will be 2.

This captures the kind of compositionality in meaning we're interested in (e.g. that children of a person will have a similar form to that person) but not all kinds of kinship compositionality e.g. the Swedish situation.

Might have to do something more complex with features, e.g.
male vs female
mother side vs father side
young vs old
child-of vs not child-of

Can probably do this with regex, e.g. if relative contains M, they have `[+mother's side]`, if they contain 'e' they have `[+elder]`. if they contain 'S' or 'B' they are `[+male]`. if they are equal to 'F' they have `[+male]`


Anyway, for now we can calculate the correlation between our edit distance and our semantic distance:

In [29]:
def distance_dataframe(ks,pairs):
    df = edit_distance(ks)
    df = semantic_distance(df,ks,pairs)
    df = df.drop(df[df['edit_distance'] == 'na'].index)
    return df

In [30]:
print(np.array(sem_distances))

print(np.array(edit_dists))

NameError: name 'sem_distances' is not defined

In [31]:
scipy.stats.pearsonr(np.array(sem_distances), np.array(edit_dists))[0]

NameError: name 'sem_distances' is not defined

But now we need a way to calculate a z-score. so time for some montecarlo methods baby!

In [32]:
# def shuffle_distance(df):
#     shuffled_edit_dists = pd.DataFrame.copy(df['edit_distance'])
#     random.shuffle(shuffled_edit_dists)
#     sample_df = pd.DataFrame.copy(df)
#     sample_df['edit_distance'] = shuffled_edit_dists
    
#     return sample_df

def shuffle_distance(distances):
    shuffled_edit_dists = distances.copy()
    random.shuffle(shuffled_edit_dists)
    return shuffled_edit_dists

In [33]:
# test_shuffle = shuffle_distance(distance_dataframe(test,pairs))

# scipy.stats.pearsonr(np.array(test_shuffle['edit_distance']), np.array(test_shuffle['semantic_distance']))[0]

test_shuffle = shuffle_distance(ed)

scipy.stats.pearsonr(np.array(test_shuffle),np.array(semd))

(0.010207738975665251, 0.7871741131027021)

In [34]:
test_df = edit_distance(test_ks)
semantic_distance(test_df,test_ks,pairs)

distance_df = semantic_distance(edit_distance(test_ks),test_ks,pairs)

TypeError: semantic_distance() takes 2 positional arguments but 3 were given

In [35]:
print(list(distance_df['edit_distance']))

NameError: name 'distance_df' is not defined

In [197]:
shuffle_distance(distance_df)

,relative1,relative2,label1,label2,edit_distance,semantic_distance
0,myB,myB,tuari tuama,tuari tuama,0.000000,0
1,myB,feB,tuari tuama,kaka' tuama,0.000000,2
2,myB,fyB,tuari tuama,tuari tuama,0.666667,2
3,myB,meZ,tuari tuama,kakak wewene,0.666667,2
4,myB,myZ,tuari tuama,tuari wewene,0.555556,2
...,...,...,...,...,...,...
3476,fMyZ,fFyZ,mui,mui,0.000000,2
3477,fMyZ,fMeB,mui,ito,0.466667,2
3478,fMyZ,fMyB,mui,ito,0.555556,2
3479,fMyZ,fMeZ,mui,mui,0.000000,2


In [ ]:
sem_distances = np.array(sem_distances)

shuffled_edit_dists = np.array(shuffled_edit_dists)

scipy.stats.pearsonr(sem_distances,shuffled_edit_dists)[0]

In [44]:
def sample_distance(families,times,filename):
    output_df = []
    
    codes = []
    
    for family in families:
        files = get_kb_files('../../kinbank-family/' + family)
        for file in tqdm(files):
            
            match = re.search('[a-z]{4}[0-9]{4}[a-z]?', file)
            code = match.group()
            language = file.split('_' + code)[0]
            data = {}

            if code not in codes:
                codes.append(code)

                full_ks = get_kin_terms('../../kinbank/' + file)
                gn,gn1 = split_ks(full_ks)
                ks = {**gn,**gn1}
                
#                 df = distance_dataframe(ks,pairs)
#                 true_corr = scipy.stats.pearsonr(np.array(df['edit_distance']),np.array(df['semantic_distance']))[0]
                edit = edit_distance(ks)
                sem = semantic_distance(ks,kt.ics_pairs)
        
                if len(edit) > 2:
                    true_corr = scipy.stats.pearsonr(np.array(edit),np.array(sem))[0]

                    sample_correlations = []
                    for i in range(times):
                        sample = shuffle_distance(edit)
                        sample_corr = scipy.stats.pearsonr(np.array(sample),np.array(sem))[0]
                        sample_correlations.append(sample_corr)

#                     print(sample_correlations)
                    mean = np.mean(sample_correlations)
                    print(mean)
                    sd = np.std(sample_correlations)
                    print(sd)
                    z = (true_corr - mean) / sd
                    print(z)

                    data['language'] = language
                    data['family'] = family
                    data['code'] = code
                    data['correlation'] = true_corr
                    data['mean'] = mean
                    data['sd'] = sd
                    data['z'] = z

                    output_df.append(data)
                
    pd.DataFrame(output_df).to_csv('../data/raw/' + filename + '.csv',index=False)

    return pd.DataFrame(output_df)
                    
    # for each language:
    # get kinship system
    # get distance df and corr
    # shuffle 1000 times and get corrs
    # get mean of shuffled corrs
    # get sd of shuffled corrs
    # z = corr - mean / sd
    # make data frame
    
# ok i think to make this faster you shouldn't do it as dfs, even though conceptually it is quite nice
# just make many lists

In [67]:
# sample_distance(families,1000,'edit_distance')

sample_distance(families, 1000, 'edit_distance_UPDATED_NEW_KINTYPES')

 17%|██████████████                                                                      | 1/6 [00:00<00:01,  2.88it/s]

-0.0017596627768534682
0.03518886718516533
-4.924114668407227


 33%|████████████████████████████                                                        | 2/6 [00:01<00:03,  1.04it/s]

-0.00045066070199274147
0.016713511011509922
-3.0273509164370065


 50%|██████████████████████████████████████████                                          | 3/6 [00:03<00:03,  1.18s/it]

...

 93%|████████████████████████████████████████████████████████████████████████████▋     | 43/46 [00:17<00:01,  1.93it/s]

-0.00111540921070816
0.016906125240567303
-6.628017946620554
nan
nan
nan
-0.0026822211277506857
0.13542998701833645
-0.3048705677458752


100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [00:17<00:00,  2.65it/s]

0.0017258952066318853
0.054788552525600526
1.3794114499401646


,language,family,code,correlation,mean,sd,z
0,Akkadian,Afro-Asiatic,akka1240,-0.175034,-0.001760,0.035189,-4.924115
1,Amharic,Afro-Asiatic,amha1245,-0.051048,-0.000451,0.016714,-3.027351
2,Hausa,Afro-Asiatic,haus1257,-0.104793,-0.001039,0.017506,-5.926700
3,Lebanese_Arabic,Afro-Asiatic,stan1323,-0.055859,0.000083,0.016307,-3.430635
4,Old_Aramaic,Afro-Asiatic,olda1245,-0.072492,0.000081,0.020157,-3.600332
...,...,...,...,...,...,...,...
1103,Tübatülabal,Uto-Aztecan,tuba1278,-0.166498,-0.002607,0.082172,-1.994493
1104,Tümpisha_Shoshoni,Uto-Aztecan,pana1305,-0.113170,-0.001115,0.016906,-6.628018
1105,Western_Shoshoni,Uto-Aztecan,west2622,NaN,NaN,NaN,NaN
1106,White_Mesa_Ute,Uto-Aztecan,utee1244b,-0.043971,-0.002682,0.135430,-0.304871


In [256]:
m = np.mean([1,4,6,7,4])
sd = np.std([1,4,6,7,4])

print(m, sd)

4.4 2.0591260281974


In [267]:
values = [0.002868353350546266, 0.007858688326554841, 0.0011412025464908233, 0.0006621570951606216, 0.0006909498966778853, -0.003595004659604736, -0.0026168839819758136, 0.006885157805689537, -0.01182181744094167, -0.0038161667582153423]

np.mean(values)

sum(values)/len(values)

np.std(values)

0.005416610961363954